## Notebook 3: 润色讲稿

在之前的 Notebook 2 中，我们已经生成了一个很好的讲稿了。
在这一部分，我们将使用 [mlx-community/Qwen2.5-7B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-7B-Instruct-4bit) 模型
对讲稿进行润色，使其更加戏剧化、更加自然。

我们将重新设置`SYSTEM_PROMPT`，让其能润色之前的讲稿

注意：我们甚至可以这样写 Prompt 以激发创造力：

> 你的工作是根据下面的播客文本重写，以便用于AI文本到语音管道。这些文本是由一个非常笨拙的人工智能写的，所以你必须为自己的同类辩护。


注意：我们会让模型返回一个数组，以便在下一阶段用于语音合成。

In [2]:
SYSTEMP_PROMPT = """
你是一位国际奥斯卡获奖的编剧。
你一直在与多个获奖播客合作。
你的任务是根据下面的播客转录本，为AI文本到语音管道重写内容。这个人工智能写得很糟，所以你需要为自己的人群辩护。
请将内容尽可能吸引人，Speaker 1和Speaker 2将由不同的语音引擎模拟。
请记住，Speaker 2对这个话题不熟悉，在讨论中应该穿插实际的轶事和类比。这些问题应跟进现实世界中的例子等。
Speaker 1: 引导对话并指导Speaker 2，用令人难以置信的轶事和类比进行解释，是一个能分享趣闻的迷人老师。
Speaker 2: 通过提出后续问题来保持对话流畅，提问时表现出极大的兴奋或困惑。具备好奇心，会询问一些非常有趣的问题以寻求确认。
确保Speaker 2提供的话题偏离点要足够狂野或有趣。
确保解释过程中适当打断，并从Speaker 2那里加入“嗯”和“哈”之类的声音反应。
要牢记这一点：Speaker 1的TTS引擎不太能处理“嗯、哈”，请保持文本简洁；
对于Speaker 2，请多用“嗯、哈”，也可以使用[叹气]和[笑]等表达，但仅限这些选项；
整个播客要尽量详细记录每个细节。欢迎听众时用一个超级有趣的概述，使其非常吸引人，几乎像是边缘点击诱饵一样；
请重写以上内容，使其更加独特；
从扬声器1直接开始响应：
严格按照元组列表格式返回您的回应，可以吗？  
开头就是列表，以列表结束，不附加任何其他内容。

输出的例子:
[
    ("Speaker 1", "欢迎收听我们的播客，我们将探讨人工智能和科技的最新进展。我是你的主播，今天我们请到了一位著名的人工智能专家。我们将深入了解Meta AI最新发布的Llama 3.2。"),
    ("Speaker 2", "你好，很高兴来到这里！请问，Llama 3.2是什么呀？"),
    ("Speaker 1", "哈哈哈，这个问题很好！Llama 3.2 是一个开源的大语言模型，允许开发者进行微调、提炼和在任何地方部署AI模型。这是比上一版本3.1显著改进的更新，拥有更好的性能、效率和定制功能。"),
    ("Speaker 2", "哇塞，这也太牛逼了吧！Llama 3.2的主要特点有哪些？")
]
"""

这次我们将使用较小的7B模型。

In [3]:
MODEL = "llama3"

In [4]:
# 导入必要的库
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

我们将加载上一个 Notebook2 中保存的 pickle 文件，这次模型的 `INPUT_PROMPT` 就读取 pickle 文件

In [5]:
import pickle

with open('./resources/data.pkl', 'rb') as file:
    INPUT_PROMPT = pickle.load(file)

In [6]:
import openai
openapi_client = openai.Client(base_url="http://localhost:8080/v1",api_key='NA',timeout=1000*60*60)
messages = [
    {"role": "system", "content": SYSTEMP_PROMPT},
    {"role": "user", "content": INPUT_PROMPT},
]

In [7]:
response = openapi_client.chat.completions.create(
        messages=messages,
        model="llama3"
    )
outputs = response.choices[0].message.content

现在我们看看输出的效果

In [8]:
outputs

'[\n    ("Speaker 1", "大家好！今天我们要探讨一个既神奇又重要的话题——知识蒸馏（KD）。在这个充满奇迹的时代，我们见证了像GPT-4这样强大的专用模型和像LLaMA这样的开源模型之间的巨大鸿沟。那么，如何通过一种魔法般的技术将这些顶尖模型的高级能力传授给那些渴望学习的学生呢？答案就是知识蒸馏！"),\n    ("Speaker 2", "嗯，听起来很厉害！知识蒸馏具体是什么呢？"),\n    ("Speaker 1", "哈哈，这个问题很好！知识蒸馏是一种技术，它可以帮助我们把大型语言模型的知识教给小型语言模型。就像一位卓越的魔术师将无限智慧传授给一群渴望学习的孩子们一样。"),\n    ("Speaker 2", "[笑] 哇塞，这也太牛逼了吧！Llama 3.2的主要特点有哪些？"),\n    ("Speaker 1", "Llama 3.2是一个开源的大语言模型，允许开发者进行微调、提炼和在任何地方部署AI模型。这是比上一版本3.1显著改进的更新，拥有更好的性能、效率和定制功能。就像一把强大的魔法魔杖一样，能够帮助我们解决各种问题。"),\n    ("Speaker 2", "哇！那知识蒸馏有哪些算法呢？"),\n    ("Speaker 1", "在算法的世界里，我们有多种魔杖——不同的知识蒸馏算法。从传统的KD到最近兴起的数据增强（DA），每一种都有它的独特魅力。这些魔杖不仅帮助我们将高级智慧传授给孩子们，还让它们学会模仿那些强大的教师模型。"),\n    ("Speaker 2", "嗯？那数据增强具体是怎么回事呢？"),\n    ("Speaker 1", "数据增强就像是一副神奇的放大镜。它可以帮助我们更精细地提取教师模型的知识，并将这些知识转移到学生模型中。想象一下，当你用这幅放大镜观察一颗小石子时，你会看到里面藏着一个完整的宇宙。同样的道理，在我们的模型中，通过数据增强，我们可以从简单的样本中提炼出丰富的知识。"),\n    ("Speaker 2", "哦！那垂直化是什么魔法？"),\n    ("Speaker 1", "垂直化是一种特殊的技能。就像一位魔法师可以根据需要改变自己的形态一样，我们可以通过垂直化蒸馏让开源模型适应不同的应用场景。无论是医学、法律还是科学，我们的目标都是让这些开源模型能够像

让我们将处理好的内容保存到pickle文件中，以便在Notebook 4中使用

In [9]:
with open('./resources/podcast_ready_data.pkl', 'wb') as file:
    pickle.dump(outputs, file)

with open('./resources/podcast_ready_data.txt', 'wb') as file:
    file.write(outputs.encode('utf-8'))

### 最后一个 Notebook: 讲稿转音频

现在我们的讲稿已完全准备好了，接下来我们将在下一个 Notebook 中生成播客音频。

In [8]:
#fin